### Single Step Tool 
    * single-step tool use scenario, an LLM calls a single tool or many tools in parallel. The tools then return the results, which the LLM uses to generate its response.

In [1]:
# pip install cohere

import json
import cohere
co = cohere.Client(api_key="j3dnErL21HLrrwoaSskJsQpmAn2SxRc1JzJWg7yL")

In [2]:
# Step 1: Create Tools

In [3]:
def daily_sales_report(day: str) -> dict:
    """
    Function to retrieve the sales report for the given day
    """
    # Mock database containing daily sales reports
    sales_database = {
    '2023-09-28': {'total_sales_amount': 5000,'total_units_sold': 100},
    '2023-09-29': {'total_sales_amount': 10000,'total_units_sold': 250},
    '2023-09-30': {'total_sales_amount': 8000,'total_units_sold': 200}
    }
    
    report = sales_database.get(day, {})
    
    if report:
        return {
            'date': day,
            'summary': f"Total Sales Amount: {report['total_sales_amount']}, Total Units Sold: {report['total_units_sold']}"
        }
    else:
        return {'date': day, 'summary': 'No sales data available for this day.'}
    

def product_database(category: str) -> dict:
    """
    Function to retrieve products for the given category
    """
    
    # Mock product catalog
    product_catalog = {
        'Electronics': [
            {'product_id': 'E1001', 'name': 'Smartphone', 'price': 500, 'stock_level': 20},
            {'product_id': 'E1002', 'name': 'Laptop', 'price': 1000, 'stock_level': 15},
            {'product_id': 'E1003', 'name': 'Tablet', 'price': 300, 'stock_level': 25},
        ],
        'Clothing': [
            {'product_id': 'C1001', 'name': 'T-Shirt', 'price': 20, 'stock_level': 100},
            {'product_id': 'C1002', 'name': 'Jeans', 'price': 50, 'stock_level': 80},
            {'product_id': 'C1003', 'name': 'Jacket', 'price': 100, 'stock_level': 40},
        ]
    }
    
    products = product_catalog.get(category, [])
    return {
        'category': category,
        'products': products
    }

functions_map = {
    "daily_sales_report": daily_sales_report,
    "product_database": product_database
}

In [4]:
# Step 2: Define Tool Schema
tools = [
    {
        "name": "daily_sales_report",
        "description": "Connects to a database to retrieve overall sales volumes and sales information for a given day.",
        "parameter_definitions": {
            "day": {
                "description": "Retrieves sales data for this day, formatted as YYYY-MM-DD.",
                "type": "str",
                "required": True
            }
        }
    },
    {
        "name": "product_database",
        "description": "A database that contains information about all the products of this company, including categories, prices, and stock levels.",
        "parameter_definitions": {
            "category": {
                "description": "Retrieves product information data for all products in this category.",
                "type": "str",
                "required": True
            }
        }
    }
]

In [5]:
# Step 3: Create Custom Preamble (Optional)
preamble = """## Task & Context
You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or similar tools to help you, which you use to research your answer. You should focus on serving the user's needs as best you can, which will be wide-ranging.

## Style Guide
Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling.
"""

In [6]:
model = "command-a-03-2025"

def run_assistant(message, chat_history=[]):
    # Step 1: Get user message
    print(f"Question:\n{message}")
    print("="*50)

    # Step 2: Generate tool calls (if any)    
    response = co.chat(
        message=message,
        model=model,
        preamble=preamble,
        tools=tools,
        chat_history=chat_history,
        force_single_step=True
    )

    while response.tool_calls:
        tool_calls = response.tool_calls
        
        if response.text:
            print("Tool plan:")
            print(response.text,"\n")
        print("Tool calls:")
        for call in tool_calls:
            print(f"Tool name: {call.name} | Parameters: {call.parameters}")
        print("="*50)
        
        # Step 3: Get tool results
        tool_results = []
        for tc in tool_calls:
            tool_call = {"name": tc.name, "parameters": tc.parameters}
            tool_output = functions_map[tc.name](**tc.parameters)
            tool_results.append({"call": tool_call, "outputs": [tool_output]})
        
        # Step 4: Generate response and citations                
        response = co.chat(
            message="",
            model=model,
            preamble=preamble,
            tools=tools,
            tool_results=tool_results,
            chat_history=response.chat_history
        )

        # Update the chat history
        chat_history = response.chat_history
        
    # Print final response
    print("Final response:")
    print(response.text)
    print("="*50)
    
    # Print citations (if any)
    if response.citations:
        print("Citations:")
        for citation in response.citations:
            print(citation)
        print("\nCited Documents:")
        for document in response.documents:
            print(document)
        print("="*50)
    
    return chat_history

In [7]:
chat_history = run_assistant("Can you provide a sales summary for 29th September 2023?")

Question:
Can you provide a sales summary for 29th September 2023?
Tool plan:
I will use one or more of the available tools to find the answer 

Tool calls:
Tool name: daily_sales_report | Parameters: {'day': '2023-09-29'}
Final response:
On 29th September 2023, the total sales amount was 10,000 and the total units sold were 250.
Citations:
start=28 end=57 text='total sales amount was 10,000' document_ids=['daily_sales_report:0:2:0'] type='TEXT_CONTENT'
start=66 end=92 text='total units sold were 250.' document_ids=['daily_sales_report:0:2:0'] type='TEXT_CONTENT'

Cited Documents:
{'date': '2023-09-29', 'id': 'daily_sales_report:0:2:0', 'summary': 'Total Sales Amount: 10000, Total Units Sold: 250'}


In [8]:
# Single-Step, Parallel Tool Use
chat_history = run_assistant("Can you provide a sales summary for 28th and 29th September 2023 as well as the stock level of the products in the 'Electronics' category?")

Question:
Can you provide a sales summary for 28th and 29th September 2023 as well as the stock level of the products in the 'Electronics' category?
Tool plan:
I will use one or more of the available tools to find the answer 

Tool calls:
Tool name: daily_sales_report | Parameters: {'day': '2023-09-28'}
Tool name: daily_sales_report | Parameters: {'day': '2023-09-29'}
Tool name: product_database | Parameters: {'category': 'Electronics'}
Final response:
Here is a sales summary for 28th and 29th September 2023:

- **28th September 2023**: Total Sales Amount: 5000, Total Units Sold: 100
- **29th September 2023**: Total Sales Amount: 10000, Total Units Sold: 250

Here is the stock level of the products in the 'Electronics' category:

- Smartphone: 20
- Laptop: 15
- Tablet: 25
Citations:
start=86 end=133 text='Total Sales Amount: 5000, Total Units Sold: 100' document_ids=['daily_sales_report:0:2:0'] type='TEXT_CONTENT'
start=161 end=209 text='Total Sales Amount: 10000, Total Units Sold: 250

## Directly Answering
 * tool use systems is the model’s ability to choose the right tools for a task. This includes the ability to decide to not use any tool, and instead, respond to a user message directly.

In [9]:
chat_history = run_assistant("Give me 3 concise tips on how to build a great company")

Question:
Give me 3 concise tips on how to build a great company
Tool plan:
I will use one or more of the available tools to find the answer 

Tool calls:
Tool name: product_database | Parameters: {'category': '3 concise tips on how to build a great company'}
Final response:
I'm sorry, I couldn't find any information on how to build a great company.


In [10]:
chat_history = run_assistant("How many employees does this company have?")

Question:
How many employees does this company have?
Tool plan:
I will use one or more of the available tools to find the answer 

Tool calls:
Tool name: product_database | Parameters: {'category': 'employees'}
Final response:
I'm sorry, I am unable to find the answer to your question.


In [11]:
# State Management (Memory)

In [12]:
# Print chat history
for turn in chat_history:
    print(turn,"\n")
    

role='USER' message='How many employees does this company have?' tool_calls=None 

role='CHATBOT' message='I will use one or more of the available tools to find the answer' tool_calls=[ToolCall(name='product_database', parameters={'category': 'employees'})] 

role='TOOL' tool_results=[ToolResult(call=ToolCall(name='product_database', parameters={'category': 'employees'}), outputs=[{'category': 'employees', 'products': []}])] 

role='CHATBOT' message="I'm sorry, I am unable to find the answer to your question." tool_calls=None 

